In [7]:
import pandas as pd

# 读取 XLSX 文件
file_path = './post_level_annotations_anonymized//Redditors_and_posts_batch_V_anonymized.xlsx'
excel_file = pd.ExcelFile(file_path)

# 创建一个空的列表来存储所有工作表的 Suicide Risk 数据
suicide_risk_data = []

# 遍历每个工作表
for sheet_name in excel_file.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # 过滤掉 'Suicide Risk' 列为空的行
    filtered_df = df[df['Suicide Risk'].notna()]
    
    # 将非空的 'Suicide Risk' 数据添加到列表中
    suicide_risk_data.extend(filtered_df['Suicide Risk'].tolist())

# 将所有工作表的 Suicide Risk 数据合并成一个 DataFrame
all_suicide_risk_df = pd.DataFrame(suicide_risk_data, columns=['Suicide Risk'])

# 统计 Suicide Risk 列中每个类别的数量
category_counts = all_suicide_risk_df['Suicide Risk'].value_counts()

# 打印结果
print(category_counts)


Suicide Risk
supportive            351
indicator             161
noninformative        129
ideations              45
ideation               19
behavior               18
attempt                13
supporitve              3
Attempt                 1
suicidal ideations      1
suppr                   1
indiccator              1
noninformtative         1
Name: count, dtype: int64


In [3]:
import pandas as pd
import glob
import os
import re
def get_all_xlsx_file_paths(folder_path):
    # 使用 glob 查找文件夹下所有的 xlsx 文件
    xlsx_files = glob.glob(os.path.join(folder_path, "*.xlsx"))
    
    # 自定义排序函数，按照文件名中的 "I", "II", "III", "IV", "V" 进行排序
    roman_numerals = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5}
    
    def extract_roman_numeral(file_name):
        match = re.search(r"batch_(I|II|III|IV|V)_", file_name)
        if match:
            return roman_numerals[match.group(1)]
        else:
            # 如果文件名中不包含罗马数字，默认排序到后面
            return float('inf')
    
    # 对文件路径进行排序
    xlsx_files.sort(key=lambda x: extract_roman_numeral(os.path.basename(x)))
    
    return xlsx_files



title = {
    'SU':['supportive','supporitive','suppr'],
    'IN':['indicator','indiccator'],
    'ID':['ideation','ideations','suicidal ideations'],
    'BE':['behavior','behaviour'],
    'AT':['attempt']
}

title1 = ['supportive','supporitive','suppr','indicator','indiccator','ideation','ideations','suicidal ideations','behavior','behaviour','attempt']


def get_category_counts(file_path, title):

    reddit_data = {
        'Suicide Risk':[],
        'Reddit Post':[]
    }

    for path in file_path:
        excel_file = pd.ExcelFile(path)
        print(path)
        for sheet_name in excel_file.sheet_names:
            # print(sheet_name)
            df = pd.read_excel(path, sheet_name=sheet_name)
            df['Suicide Risk'].fillna('nan', inplace=True)
            risk_levels = df['Suicide Risk'].values

            risk_posts = df['Reddit Post'].values
            print(risk_levels)
            for i,risk in enumerate(risk_levels):
                if risk.lower() in title:
                    reddit_data['Suicide Risk'].append(risk)
                    reddit_data['Reddit Post'].append(risk_posts[i])


    # 将reddit_data转化为df对象
    reddit_df = pd.DataFrame(reddit_data)
    reddit_df.to_csv('./data/post_level.csv',index=False)

         





folde_path = './post_level_annotations_anonymized'
paths = get_all_xlsx_file_paths(folde_path)
get_category_counts(paths,title1)

    
    
                                      
        


./post_level_annotations_anonymized\Redditors_and_posts_batch_I_anonymized.xlsx
['supportive' 'ideation' 'indicator' 'supportive' 'supportive' 'indicator'
 'indicator' 'supportive' 'ideation' 'ideation']
['supportive']
['supportive' 'supportive' 'supportive' 'supportive' 'supportive'
 'supportive' 'supportive' 'supportive']
['ideation' 'supportive' 'supportive' 'supportive']
['supportive' 'attempt' 'behavior' 'nan' 'nan' 'nan']
['uninformative' 'supportive' 'uninformative' 'ideation' 'supportive'
 'supportive' 'supportive' 'supportive' 'ideation' 'supportive'
 'uninformative' 'supportive' 'supportive' 'uninformative' 'indicator'
 'supportive' 'supportive' 'supportive']
['uninformative' 'uninformative' 'ideation' 'uninformative' 'ideation'
 'supportive']
['supportive' 'supportive' 'supportive' 'supportive' 'supportive'
 'supportive' 'supportive' 'supportive' 'supportive' 'supportive'
 'supportive' 'uninformative' 'supportive' 'supportive' 'supportive'
 'supportive' 'supportive' 'support

In [27]:
import os
import pandas as pd

def read_all_xlsx_files(folder_path, title):
    # 获取文件夹中的所有文件
    files = os.listdir(folder_path)
    
    # 过滤出所有xlsx文件
    xlsx_files = [file for file in files if file.endswith('.xlsx')]
    
    # 创建一个空的POST对象
    POST = {key: [] for key in title.keys()}
    
    # 遍历所有xlsx文件
    for xlsx_file in xlsx_files:
        file_path = os.path.join(folder_path, xlsx_file)
        # 读取当前xlsx文件中的所有表
        xls = pd.ExcelFile(file_path)
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            if 'Suicide Risk' in df.columns and 'Reddit Post' in df.columns:
                for index, row in df.iterrows():
                    suicide_risk = row['Suicide Risk']
                    reddit_post = row['Reddit Post']
                    for key, values in title.items():
                        if any(suicide_risk.lower() == value.lower() for value in values):
                            POST[key].append(reddit_post)
                            break  # 如果找到匹配的值，跳出循环
    
    return POST

# 定义title对象
title = {
    'SU': ['supportive', 'supporitive', 'suppr'],
    'IN': ['indicator', 'indiccator'],
    'ID': ['ideation', 'ideations', 'suicidal ideations'],
    'BE': ['behavior', 'behaviour'],
    'AT': ['attempt']
}

# 使用示例
folder_path = './post_level_annotations_anonymized/'  # 替换为实际的文件夹路径
post_data = read_all_xlsx_files(folder_path, title)

# 打印POST对象
for key, values in post_data.items():
    print(f"{key}: {values}")


AttributeError: 'float' object has no attribute 'lower'

In [1]:
# 开始划分训练集，测试集

import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import emoji
import re
# 读取csv文件

df = pd.read_csv('./data/post_level.csv')

X = df['Reddit Post']
y = df['Suicide Risk']

def preprocess_reddit_text(text):

    text = emoji.demojize(text)
    # 将HTML实体替换为常规字符
    text = text.replace('&gt;', '>').replace('&lt;', '<').replace('&amp;', '&')
    
    # 去除特定的标记
    text = re.sub(r'\[deleted\]|\[removed\]', '', text)
    
    # 将链接替换为占位符
    text = re.sub(r'http\S+|www\S+|https\S+', 'URL', text, flags=re.MULTILINE)
    
    # 将表情符号转换为文本描述

    # 去除 @ 和 # 开头的标签
    text = re.sub(r'@\w+|#\w+', '', text)

    text = re.sub(r'\d+', 'number', text)

        # 去除标点符号和其他特殊字符
    text = re.sub(r'[^\w\s]', '', text)
    
    # 去除多余的空格
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 将文本转换为小写并进行词标记化
    text = text.lower()
    # tokens = word_tokenize(text)
    # text = ' '.join(tokens)
    return text

# 使用train_test_split进行划分
# 第一次划分，将数据划分为80%的训练集和20%的测试集
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=24, stratify=y)

# 第二次划分，将80%的数据再划分为75%的训练集和25%的验证集，最终得到6:2:2的比例
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=24, stratify=y_train_val)



# print(y_train[:10]

# with open('post.txt', 'w') as file:
#     # 写入训练集
#     for idx in range(len(X_train_val)):
#         label = y_train_val.iloc[idx]
#         file.write(f'{idx} \t train \t {label}\n')

#     # for idx in range(len(X_val)):
#     #     label = y_val.iloc[idx]
#     #     file.write(f'{len(X_train) + idx} \t val \t {label}\n')
    
#     # 写入测试集
#     for idx in range(len(X_test)):
#         label = y_test.iloc[idx]
#         train_val = len(X_train_val)
#         file.write(f'{train_val+ idx} \t test \t {label}\n')


# with open('post_clean.txt', 'w') as file:
#     # 写入训练集特征
#     for idx in range(len(X_train_val)):
#         feature = X_train_val.iloc[idx]
#         feature = preprocess_reddit_text(feature)
#         file.write(f'{feature}\n')

#     # for idx in range(len(X_val)):
#     #     feature = X_val.iloc[idx]
#     #     feature = preprocess_reddit_text(feature)
#     #     file.write(f'{feature}\n')
    
#     # 写入测试集特征
#     for idx in range(len(X_test)):
#         feature = X_test.iloc[idx]
#         feature = preprocess_reddit_text(feature)
#         file.write(f'{feature}\n')

with open('post.txt', 'w') as file:
    # 写入训练集
    for idx in range(len(X_train)):
        label = y_train.iloc[idx]
        file.write(f'{idx} \t train \t {label}\n')

    for idx in range(len(X_val)):
        label = y_val.iloc[idx]
        file.write(f'{len(X_train) + idx} \t val \t {label}\n')
    
    # 写入测试集
    for idx in range(len(X_test)):
        label = y_test.iloc[idx]
        train_val = len(X_train) + len(X_val)
        file.write(f'{train_val+ idx} \t test \t {label}\n')

# 写入X_train和X_test内容到features.txt文件
with open('post.clean.txt', 'w') as file:
    # 写入训练集特征
    for idx in range(len(X_train)):
        feature = X_train.iloc[idx]
        feature = preprocess_reddit_text(feature)
        file.write(f'{feature}\n')

    for idx in range(len(X_val)):
        feature = X_val.iloc[idx]
        feature = preprocess_reddit_text(feature)
        file.write(f'{feature}\n')
    
    # 写入测试集特征
    for idx in range(len(X_test)):
        feature = X_test.iloc[idx]
        feature = preprocess_reddit_text(feature)
        file.write(f'{feature}\n')

print("训练集和测试集特征已保存到 features.txt 文件中")


# 将长度大于512找到

# cout = 1
# for idx in range(len(X)):
#     feature = X.iloc[idx]
#     if len(feature) > 1024:
#         cout += 1
# print(cout)

# 计算平均长度
# total_length = 0
# for idx in range(len(X)):
#     total_length += len(re.sub(r'[^\w\s]', '', X.iloc[idx]))

# average_length = total_length / len(X)
# print(f"平均长度: {average_length}")




训练集和测试集特征已保存到 features.txt 文件中


In [5]:
# 读取数据
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import emoji
import re
from nltk import pos_tag
import nltk
import numpy as np
from nltk.corpus import wordnet
# import nltk



def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN





# 读取txt文件
def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines

lemmatizer = nltk.stem.WordNetLemmatizer()
data = read_txt_file('./clean_text/post.clean.txt')

for i in range(10):
  pos_tagged = pos_tag(nltk.word_tokenize(data[i]))
  word_list = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tagged]
  print(word_list)
    


['i', 'think', 'you', 'be', 'probably', 'suffer', 'from', 'depression', 'if', 'your', 'insurancemonetary', 'state', 'allow', 'it', 'try', 'to', 'go', 'see', 'a', 'doctor', 'to', 'be', 'test', 'if', 'not', 'just', 'talk', 'to', 'people', 'about', 'your', 'problem', 'friends', 'parent', 'sibling', 'etc', 'or', 'if', 'not', 'shoot', 'me', 'a', 'pm', 'i', 'may', 'not', 'have', 'the', 'right', 'answer', 'but', 'if', 'you', 'need', 'to', 'im', 'right', 'here']
['if', 'she', 'know', 'how', 'broken', 'you', 'be', 'now', 'she', 'would', 'be', 'devastate']
['not', 'gon', 'na', 'lie', 'it', 'not', 'as', 'easy', 'a', 'stop', 'feeling', 'worthless', 'then', 'it', 'will', 'take', 'time', 'but', 'talk', 'to', 'people', 'good', 'friend', 'parent', 'sibling', 'or', 'even', 'people', 'online', 'that', 'be', 'willing', 'to', 'try', 'and', 'help', 'people', 'if', 'you', 'have', 'a', 'good', 'doctor', 'then', 'talk', 'to', 'them', 'and', 'try', 'to', 'get', 'some', 'antidepressant']
['hey', 'so', 'i', 'jus

# # # 开始训练帖子的bert语义

In [8]:
import pandas as pd
# 开始训练得到文本的嵌入表示---利用bert
from transformers import BertTokenizer, BertModel
import torch
import pickle



# 读取txt文件
def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines


data = read_txt_file('./clean_text/post.clean.txt')

# 用bert对帖子进行语义编码

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    # 将文本转换为模型输入格式
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # 提取[CLS] token的嵌入表示
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    # embedding = embeddings.cpu().numpy()
    return embedding




# 获取每个帖子的BERT嵌入，并存储到pkl文件中
embeddings = []
for text in data:
    embedding = get_bert_embedding(text)
    embeddings.append(embedding)

# 将所有帖子的嵌入表示保存到pkl文件中
with open('./data/post_embeddings.pkl', 'wb') as file:
    pickle.dump(embeddings, file)

print("帖子的BERT嵌入表示已保存到 post_embeddings.pkl 文件中")

帖子的BERT嵌入表示已保存到 post_embeddings.pkl 文件中


In [11]:

import pickle
import numpy as np

# 加载帖子的BERT嵌入表示
with open('./data/post_embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

embeddings = np.array(embeddings)

# 将维度变化为300



帖子的嵌入表示维度: (200, 768)
